In [ ]:
import utils
import importlib
import pandas as pd
importlib.reload(utils)

models = {
    'kimi': 'moonshotai/kimi-k2:free',
    'qwen-8b': 'qwen/qwen3-8b:free',
    'deepseek-qwen': ' deepseek/deepseek-r1-0528-qwen3-8b:free',
    'deepseek': 'deepseek/deepseek-r1-0528:free',
}

In [45]:
df = pd.read_csv('data/train.csv')
df

,task,answer
0,The value of $y$ varies inversely as $\sqrt x$...,[600]
1,In how many ways can 4 distinct beads be place...,[12]
2,In how many ways can seven beads of distinct c...,[420]
3,"Find the number of ordered quadruples $(a,b,c,...",[8]
4,Точки \(A\) и \(C\) разбивают окружность н...,[100.0]
...,...,...
995,How many zeros are at the end of (100!)(200!)(...,[147]
996,What is the sum of the squares of all real val...,[20]
997,"The average (mean) of 20 numbers is 30, and th...",[24]
998,57+70=,[127]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

def evaluate(prompts, answers, models):

    results = {
        'model': [],
        'accuracy': [],
        'total': len(prompts),
        'correct': [],
        'predictions': {}
    }
    
    # true_answers = []
    # for ans in answers:
    #     if isinstance(ans, str):
    #         if '/' in ans:
    #             num, denom = map(float, ans.split('/'))
    #             true_answers.append(num / denom)
    #         else:
    #             true_answers.append(float(ans))
    #     else:
    #         true_answers.append(float(ans))
    
    #for model_name, model_code in tqdm(models.items(), desc="Evaluating models"):
    for model_name, model_code in models.items():
        model_predictions = []
        model_correct = 0
        
        for prompt in tqdm(prompts, desc=f"Processing {model_name}", leave=False):
            raw_answer = utils.get_answer_from_api(model_name=model_code, content=prompt)
            prediction = utils.extract_answer(raw_answer)
            
            try: 
            #вроде все ответы от модельки нормальные, но лучше перебдеть
                if isinstance(prediction, str):
                    if '/' in prediction:
                        num, denom = map(float, prediction.split('/'))
                        pred_float = num / denom
                    else:
                        pred_float = float(prediction)
                else:
                    pred_float = prediction
            except Exception as e:
                print(e)
                pred_float = float('nan')
            
            model_predictions.append(pred_float)
        
        for i, pred in enumerate(model_predictions):
            if not np.isnan(pred) and abs(pred - answers[i]) < 1e-5:
                model_correct += 1
        
        accuracy = model_correct / len(prompts)
        results['model'].append(model_name)
        results['accuracy'].append(accuracy)
        results['correct'].append(model_correct)
        results['predictions'][model_name] = model_predictions
    
    return results

def visualize_results(results):
    df = pd.DataFrame({
        'Model': results['model'],
        'Accuracy': results['accuracy'],
        'Correct': results['correct'],
        'Total': results['total']
    })
    
    df = df.sort_values('Accuracy', ascending=False)
    

    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    ax = sns.barplot(x='Accuracy', y='Model', data=df, palette='viridis')
    plt.title('Accuracy Comparison')
    plt.xlabel('Accuracy')
    plt.ylabel('Model')
    
    for i, v in enumerate(df['Accuracy']):
        ax.text(v + 0.01, i, f"{v:.4f}", color='black', va='center')

    plt.subplot(1, 2, 2)
    ax = sns.barplot(x='Correct', y='Model', data=df, palette='rocket')
    plt.title('Correct Answers Count')
    plt.xlabel('Correct Answers')
    plt.ylabel('')
    plt.tight_layout()
    
    for i, v in enumerate(df['Correct']):
        ax.text(v + 0.5, i, f"{v}/{results['total']}", color='black', va='center')
    
    plt.show()
    
    return df

prompts = df['task']
answers = df['answer']
results = evaluate(prompts, answers, models)

df_results = visualize_results(results)
print(df_results)

Processing kimi:   0%|          | 1/1000 [00:05<1:37:47,  5.87s/it]

Критическая ошибка: RateLimitError - Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1752883200000'}, 'provider_name': None}}, 'user_id': 'user_302lpaqTV9wiFmftAdqYwxtpGww'}
could not convert string to float: 'ERROR: RateLimitError'


Processing kimi:   0%|          | 2/1000 [00:11<1:31:02,  5.47s/it]

Критическая ошибка: RateLimitError - Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1752883200000'}, 'provider_name': None}}, 'user_id': 'user_302lpaqTV9wiFmftAdqYwxtpGww'}
could not convert string to float: 'ERROR: RateLimitError'


KeyboardInterrupt: 